In [4]:
import requests
from xml.etree import ElementTree as ET

response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=13')
print(response)
tree = ET.fromstring(response.content)

<Response [200]>


In [5]:
# List of all the elements
#[elem.tag for elem in tree.iter()]

In [6]:
# See the whole text in XML format
#print(ET.tostring(tree, encoding='utf8').decode('utf8'))

### The max number of pages on the API is 283939

The number after "id=" in the website name below goes from 0 to 283939. However, page 0 doesn't contain any data so I skip it in the coding below.

https://www.boardgamegeek.com/xmlapi2/thing?id=1 



In [7]:
# Code to download the pertinent info from the boardgame page

if tree[0].attrib['type'] == 'boardgame':
    boardgamecategory = []
    boardgamemechanic = []
    for i in range(len(tree[0])):
        if tree[0][i].tag == 'name':
            if tree[0][i].attrib['type'] == 'primary':
                boardgame_name = tree[0][i].attrib['value']
        elif tree[0][i].tag == 'yearpublished':
            year_publ = tree[0][i].attrib['value']
        elif tree[0][i].tag == 'minplayers':
            minplayers = tree[0][i].attrib['value']
        elif tree[0][i].tag == 'maxplayers':
            maxplayers = tree[0][i].attrib['value']     
        elif tree[0][i].tag == 'playingtime':
            playingtime = tree[0][i].attrib['value'] 
        elif tree[0][i].tag == 'minplaytime':
            minplaytime = tree[0][i].attrib['value'] 
        elif tree[0][i].tag == 'maxplaytime':
            maxplaytime = tree[0][i].attrib['value']
        elif tree[0][i].tag == 'link':
            if tree[0][i].attrib['type'] == 'boardgamecategory':
                boardgamecategory.append(tree[0][i].attrib['value'])
            elif tree[0][i].attrib['type'] == 'boardgamemechanic':
                boardgamemechanic.append(tree[0][i].attrib['value'])

    
    print('boardgame: ', boardgame_name)
    print('year published: ', year_publ)
    print('min players: ', minplayers)
    print('max players: ', maxplayers)
    print('playing time: ', playingtime, 'minutes')
    print('min playing time: ', minplaytime, 'minutes')
    print('max playing time: ', maxplaytime, 'minutes')
    print('category: ', boardgamecategory)
    print('boardgame mechanic: ', boardgamemechanic)
    

boardgame:  Catan
year published:  1995
min players:  3
max players:  4
playing time:  120 minutes
min playing time:  60 minutes
max playing time:  120 minutes
category:  ['Negotiation']
boardgame mechanic:  ['Dice Rolling', 'Modular Board', 'Route/Network Building', 'Trading']


In [8]:
# Creating a function to get the boardgame data from each page

def boardgame_info(tree):
    if tree[0].attrib['type'] == 'boardgame':
        boardgamecategory = []
        boardgamemechanic = []
        for i in range(len(tree[0])):
            if tree[0][i].tag == 'name':
                if tree[0][i].attrib['type'] == 'primary':
                    boardgame_name = tree[0][i].attrib['value']
            elif tree[0][i].tag == 'yearpublished':
                year_publ = tree[0][i].attrib['value']
            elif tree[0][i].tag == 'minplayers':
                minplayers = tree[0][i].attrib['value']
            elif tree[0][i].tag == 'maxplayers':
                maxplayers = tree[0][i].attrib['value']     
            elif tree[0][i].tag == 'playingtime':
                playingtime = tree[0][i].attrib['value'] 
            elif tree[0][i].tag == 'minplaytime':
                minplaytime = tree[0][i].attrib['value'] 
            elif tree[0][i].tag == 'maxplaytime':
                maxplaytime = tree[0][i].attrib['value']
            elif tree[0][i].tag == 'link':
                if tree[0][i].attrib['type'] == 'boardgamecategory':
                    boardgamecategory.append(tree[0][i].attrib['value'])
                elif tree[0][i].attrib['type'] == 'boardgamemechanic':
                    boardgamemechanic.append(tree[0][i].attrib['value'])
        return boardgame_name, year_publ, minplayers, maxplayers, playingtime, minplaytime, maxplaytime, boardgamecategory, boardgamemechanic

In [9]:
# Testing to make sure the function works

boardgame_info(tree)

('Catan',
 '1995',
 '3',
 '4',
 '120',
 '60',
 '120',
 ['Negotiation'],
 ['Dice Rolling', 'Modular Board', 'Route/Network Building', 'Trading'])

In [10]:
# Looping through the first several pages and getting all the necessary boardgame info with the function boardgame_info()

site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
page_num = 0
for i in range(14):
    if i > 0:
        page_num = i
        website = site+str(page_num)
        response = requests.get(website)
        tree = ET.fromstring(response.content)
        #do this only for the ones that = 'boardgame'
        if len(tree) > 0 and tree[0].attrib['type'] == 'boardgame':
            print(boardgame_info(tree), '\n')

('Die Macher', '1986', '3', '5', '240', '240', '240', ['Economic', 'Negotiation', 'Political'], ['Area Control / Area Influence', 'Auction/Bidding', 'Dice Rolling', 'Hand Management', 'Simultaneous Action Selection']) 

('Dragonmaster', '1981', '3', '4', '30', '30', '30', ['Card Game', 'Fantasy'], ['Trick-taking']) 

('Samurai', '1998', '2', '4', '60', '30', '60', ['Abstract Strategy', 'Medieval'], ['Area Control / Area Influence', 'Hand Management', 'Set Collection', 'Tile Placement']) 

('Tal der Könige', '1992', '2', '4', '60', '60', '60', ['Ancient'], ['Action Point Allowance System', 'Area Control / Area Influence', 'Auction/Bidding', 'Set Collection']) 

('Acquire', '1964', '2', '6', '90', '90', '90', ['Economic'], ['Hand Management', 'Stock Holding', 'Tile Placement']) 

('Mare Mediterraneum', '1989', '2', '6', '240', '240', '240', ['Civilization', 'Nautical'], ['Dice Rolling']) 

('Cathedral', '1978', '2', '2', '20', '20', '20', ['Abstract Strategy'], ['Area Enclosure', 'Patter

In [ ]:
# Test to see if I can loop through every boardgame page of the API 

site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
page_num = 0
count = 0
for i in range(283940):
    if i > 0:
        page_num = i
        website = site+str(page_num)
        response = requests.get(website)
        tree = ET.fromstring(response.content)
        #do this only for the ones that = 'boardgame'
        #if len(tree) > 0 and tree[0].attrib['type'] == 'boardgame':
        count += 1
            #print(i)
print(count)

# This code failed. Error message below

### The below is the error received after running the previous cell

---------------------------------------------------------------------------
TimeoutError                              Traceback (most recent call last)
~\Anaconda3\lib\site-packages\urllib3\connection.py in _new_conn(self)
    140             conn = connection.create_connection(
--> 141                 (self.host, self.port), self.timeout, **extra_kw)
    142 

~\Anaconda3\lib\site-packages\urllib3\util\connection.py in create_connection(address, timeout, source_address, socket_options)
     82     if err is not None:
---> 83         raise err
     84 

~\Anaconda3\lib\site-packages\urllib3\util\connection.py in create_connection(address, timeout, source_address, socket_options)
     72                 sock.bind(source_address)
---> 73             sock.connect(sa)
     74             return sock

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

NewConnectionError                        Traceback (most recent call last)
~\Anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    600                                                   body=body, headers=headers,
--> 601                                                   chunked=chunked)
    602 

~\Anaconda3\lib\site-packages\urllib3\connectionpool.py in _make_request(self, conn, method, url, timeout, chunked, **httplib_request_kw)
    345         try:
--> 346             self._validate_conn(conn)
    347         except (SocketTimeout, BaseSSLError) as e:

~\Anaconda3\lib\site-packages\urllib3\connectionpool.py in _validate_conn(self, conn)
    849         if not getattr(conn, 'sock', None):  # AppEngine might not have  `.sock`
--> 850             conn.connect()
    851 

~\Anaconda3\lib\site-packages\urllib3\connection.py in connect(self)
    283         # Add certificate verification
--> 284         conn = self._new_conn()
    285 

~\Anaconda3\lib\site-packages\urllib3\connection.py in _new_conn(self)
    149             raise NewConnectionError(
--> 150                 self, "Failed to establish a new connection: %s" % e)
    151 

NewConnectionError: <urllib3.connection.VerifiedHTTPSConnection object at 0x000001A95C484E10>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

MaxRetryError                             Traceback (most recent call last)
~\Anaconda3\lib\site-packages\requests\adapters.py in send(self, request, stream, timeout, verify, cert, proxies)
    439                     retries=self.max_retries,
--> 440                     timeout=timeout
    441                 )

~\Anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    638             retries = retries.increment(method, url, error=e, _pool=self,
--> 639                                         _stacktrace=sys.exc_info()[2])
    640             retries.sleep()

~\Anaconda3\lib\site-packages\urllib3\util\retry.py in increment(self, method, url, response, error, _pool, _stacktrace)
    387         if new_retry.is_exhausted():
--> 388             raise MaxRetryError(_pool, url, error or ResponseError(cause))
    389 

MaxRetryError: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /xmlapi2/thing?id=55175 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001A95C484E10>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

During handling of the above exception, another exception occurred:

ConnectionError                           Traceback (most recent call last)
<ipython-input-38-bcbfb294685c> in <module>()
      6         page_num = i
      7         website = site+str(page_num)
----> 8         response = requests.get(website)
      9         tree = ET.fromstring(response.content)
     10         #do this only for the ones that = 'boardgame'

~\Anaconda3\lib\site-packages\requests\api.py in get(url, params, **kwargs)
     70 
     71     kwargs.setdefault('allow_redirects', True)
---> 72     return request('get', url, params=params, **kwargs)
     73 
     74 

~\Anaconda3\lib\site-packages\requests\api.py in request(method, url, **kwargs)
     56     # cases, and look like a memory leak in others.
     57     with sessions.Session() as session:
---> 58         return session.request(method=method, url=url, **kwargs)
     59 
     60 

~\Anaconda3\lib\site-packages\requests\sessions.py in request(self, method, url, params, data, headers, cookies, files, auth, timeout, allow_redirects, proxies, hooks, stream, verify, cert, json)
    506         }
    507         send_kwargs.update(settings)
--> 508         resp = self.send(prep, **send_kwargs)
    509 
    510         return resp

~\Anaconda3\lib\site-packages\requests\sessions.py in send(self, request, **kwargs)
    616 
    617         # Send the request
--> 618         r = adapter.send(request, **kwargs)
    619 
    620         # Total elapsed time of the request (approximately)

~\Anaconda3\lib\site-packages\requests\adapters.py in send(self, request, stream, timeout, verify, cert, proxies)
    506                 raise SSLError(e, request=request)
    507 
--> 508             raise ConnectionError(e, request=request)
    509 
    510         except ClosedPoolError as e:

ConnectionError: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): 
        Max retries exceeded with url: /xmlapi2/thing?id=55175 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001A95C484E10>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))
